In [32]:
import numpy as np
from astropy.io import ascii
from scipy.optimize import root_scalar
from astropy import units as u
from astropy import constants as const
from astropy.time import Time
import matplotlib.pyplot as plt

# Simbolos y constantes 
"""
a = semieje mayor
e = excentricidad
omega = longitud del nodo ascendente
w = argumento del pericentro
i = inclinacion de la orbita
n = movimiento medio
lo = anomalia media en la epoca
tp = tiempo del perihelio
to = tiempo de la epoca
M = la suma de las masas de los cuerpos
"""


# Efemerides Tierra
a_tierra = (1.495582533630905E+8*u.km).to('au')
ecc_tierra = 1.694863932474438E-02
omega_tierra = 1.498625963929686E+02*u.deg
w_tierra = 3.146587763491455E+02*u.deg
i_tierra = 4.164075603038432E-03*u.deg 
n_tierra = 1.141204629731537E-05*u.deg/u.s
l0_tierra = 1.817846947871890E+02*u.deg

#Transformacion de unidades
i_tierra = i_tierra.to('rad')
omega_tierra = omega_tierra.to('rad')
w_tierra = w_tierra.to('rad')
n_tierra = n_tierra.to('rad/s')
l0_tierra = l0_tierra.to('rad')


# Ecuacion de kepler
def Kepler(E, l, ecc):
    f = E - ecc*np.sin(E) - l
    fprime = 1 - ecc*np.cos(E)
    return f, fprime


def iota(t, l0, n, t0,):
    return (l0 + n*(t-t0))


def find_f(e, E):
    root_term = np.sqrt( ((1+e)/(1-e)) )
    right_side = root_term*np.tan(E/2)
    f = 2*np.arctan(right_side)
    return f


def find_phi(f, w):
    return f+w


def find_r(a, ecc, E):
    return a*(1 - ecc*np.cos(E))


def find_xyz (r, Om, f, w, i):
    x = r*(np.cos(Om)*np.cos(f+w)-np.cos(i)*np.sin(Om)*np.sin(f+w))
    y = r*(np.sin(Om)*np.cos(f+w)-np.cos(i)*np.cos(Om)*np.sin(f+w))
    z = r*(np.sin(i)*np.sin(f+w))
    return (x, y, z)


# Orbita de la tierra
t0 = Time('2023-07-09T00:00:00', scale='tdb', format = 'isot')
t0 = t0.utc

# Provisional
t = Time('2023-07-09T00:00:00', scale='tdb', format = 'isot')
t = t.utc

## 1. Hallar iota
iota_tierra = iota(t, l0_tierra, n_tierra, t0)


## 2. Solucionar la Ec. de Kepler
# E_raiz_tierra = root_scalar(Kepler, args=(iota_tierra.value, ecc_tierra), bracket=[0, iota_tierra.value], method="brentq")
E_raiz_tierra = root_scalar(Kepler, args=(iota_tierra.value, ecc_tierra), fprime = True, x0 = 0, xtol = 1E-8).root*u.rad


## 3. Hallar f
f_tierra = find_f(ecc_tierra, E_raiz_tierra)
print(f_tierra)


## 4. Hallar phi
phi_tierra = find_phi(f_tierra, w_tierra)


## 5. Hallar r
r_tierra = find_r(a_tierra, ecc_tierra, E_raiz_tierra)

## 6. Hallar x, y, z
coor_tierra = find_xyz (r_tierra, omega_tierra, f_tierra, w_tierra, i_tierra)



-3.111477633229183 rad


TypeError: only dimensionless scalar quantities can be converted to Python scalars

In [ ]:
(r_tierra - 1.01667137050578*u.AU).to("km")